In [1]:
#! /usr/bin/env python3
#
# %BANNER_BEGIN%
# ---------------------------------------------------------------------
# %COPYRIGHT_BEGIN%
#
#  Magic Leap, Inc. ("COMPANY") CONFIDENTIAL
#
#  Unpublished Copyright (c) 2020
#  Magic Leap, Inc., All Rights Reserved.
#
# NOTICE:  All information contained herein is, and remains the property
# of COMPANY. The intellectual and technical concepts contained herein
# are proprietary to COMPANY and may be covered by U.S. and Foreign
# Patents, patents in process, and are protected by trade secret or
# copyright law.  Dissemination of this information or reproduction of
# this material is strictly forbidden unless prior written permission is
# obtained from COMPANY.  Access to the source code contained herein is
# hereby forbidden to anyone except current COMPANY employees, managers
# or contractors who have executed Confidentiality and Non-disclosure
# agreements explicitly covering such access.
#
# The copyright notice above does not evidence any actual or intended
# publication or disclosure  of  this source code, which includes
# information that is confidential and/or proprietary, and is a trade
# secret, of  COMPANY.   ANY REPRODUCTION, MODIFICATION, DISTRIBUTION,
# PUBLIC  PERFORMANCE, OR PUBLIC DISPLAY OF OR THROUGH USE  OF THIS
# SOURCE CODE  WITHOUT THE EXPRESS WRITTEN CONSENT OF COMPANY IS
# STRICTLY PROHIBITED, AND IN VIOLATION OF APPLICABLE LAWS AND
# INTERNATIONAL TREATIES.  THE RECEIPT OR POSSESSION OF  THIS SOURCE
# CODE AND/OR RELATED INFORMATION DOES NOT CONVEY OR IMPLY ANY RIGHTS
# TO REPRODUCE, DISCLOSE OR DISTRIBUTE ITS CONTENTS, OR TO MANUFACTURE,
# USE, OR SELL ANYTHING THAT IT  MAY DESCRIBE, IN WHOLE OR IN PART.
#
# %COPYRIGHT_END%
# ----------------------------------------------------------------------
# %AUTHORS_BEGIN%
#
#  Originating Authors: Paul-Edouard Sarlin
#                       Daniel DeTone
#                       Tomasz Malisiewicz
#
# %AUTHORS_END%
# --------------------------------------------------------------------*/
# %BANNER_END%

from pathlib import Path
import argparse
import random
import numpy as np
import matplotlib.cm as cm
import torch


from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

torch.set_grad_enabled(False)

info_pairs = 'test.txt'
input_dir = 'test'

with open(info_pairs, 'r') as f:
    pairs = [l.split() for l in f.readlines()]

device = 'cuda' if torch.cuda.is_available() and not None else 'cpu'
print('Running inference on device \"{}\"'.format(device))

config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024
    },
    'superglue': {
        'weights': 'outdoor',
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
    }
}

matching = Matching(config).eval().to(device)

# Create the output directories if they do not exist already.
input_dir = Path(input_dir)
print('Looking for data in directory \"{}\"'.format(input_dir))
output_dir = Path('out')
output_dir.mkdir(exist_ok=True, parents=True)
print('Will write matches to directory \"{}\"'.format(output_dir))
#if opt.eval:
#    print('Will write evaluation results',
#            'to directory \"{}\"'.format(output_dir))

print('Will write visualization images to',
            'directory \"{}\"'.format(output_dir))

timer = AverageTimer(newline=True)
for i, pair in enumerate(pairs):
        name0, name1 = pair[:2]
        stem0, stem1 = Path(name0).stem, Path(name1).stem
        matches_path = output_dir / '{}_{}_matches.npz'.format(stem0, stem1)
        eval_path = output_dir / '{}_{}_evaluation.npz'.format(stem0, stem1)
        viz_path = output_dir / '{}_{}_matches.{}'.format(stem0, stem1, 'png')
        viz_eval_path = output_dir / \
            '{}_{}_evaluation.{}'.format(stem0, stem1, 'png')

        # Handle --cache logic.
        do_match = True
        do_eval = False
        do_viz = True
        do_viz_eval = False


        if not (do_match or do_eval or do_viz or do_viz_eval):
            timer.print('Finished pair {:5} of {:5}'.format(i, len(pairs)))
            continue

        # If a rotation integer is provided (e.g. from EXIF data), use it:
        if len(pair) >= 5:
            rot0, rot1 = int(pair[2]), int(pair[3])
        else:
            rot0, rot1 = 0, 0

        # Load the image pair.
        image0, inp0, scales0 = read_image(
            input_dir / name0, device, [640, 480], rot0, None)
        image1, inp1, scales1 = read_image(
            input_dir / name1, device, [640, 480], rot1, None)
        if image0 is None or image1 is None:
            print('Problem reading image pair: {} {}'.format(
                input_dir/name0, input_dir/name1))
            exit(1)
        timer.update('load_image')

        if do_match:
            # Perform the matching.
            pred = matching({'image0': inp0, 'image1': inp1})
            pred = {k: v[0].cpu().numpy() for k, v in pred.items()}

            #### try this to change with our features
            kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
            matches, conf = pred['matches0'], pred['matching_scores0']

            
            timer.update('matcher')

            # Write the matches to disk.
            out_matches = {'keypoints0': kpts0, 'keypoints1': kpts1,
                           'matches': matches, 'match_confidence': conf}
            np.savez(str(matches_path), **out_matches)

        # Keep the matching keypoints.
        valid = matches > -1
        mkpts0 = kpts0[valid]
        mkpts1 = kpts1[matches[valid]]
        mconf = conf[valid]

        if do_eval:
            # Estimate the pose and compute the pose error.
            assert len(pair) == 38, 'Pair does not have ground truth info'
            K0 = np.array(pair[4:13]).astype(float).reshape(3, 3)
            K1 = np.array(pair[13:22]).astype(float).reshape(3, 3)
            T_0to1 = np.array(pair[22:]).astype(float).reshape(4, 4)

            # Scale the intrinsics to resized image.
            K0 = scale_intrinsics(K0, scales0)
            K1 = scale_intrinsics(K1, scales1)

            # Update the intrinsics + extrinsics if EXIF rotation was found.
            if rot0 != 0 or rot1 != 0:
                cam0_T_w = np.eye(4)
                cam1_T_w = T_0to1
                if rot0 != 0:
                    K0 = rotate_intrinsics(K0, image0.shape, rot0)
                    cam0_T_w = rotate_pose_inplane(cam0_T_w, rot0)
                if rot1 != 0:
                    K1 = rotate_intrinsics(K1, image1.shape, rot1)
                    cam1_T_w = rotate_pose_inplane(cam1_T_w, rot1)
                cam1_T_cam0 = cam1_T_w @ np.linalg.inv(cam0_T_w)
                T_0to1 = cam1_T_cam0

            epi_errs = compute_epipolar_error(mkpts0, mkpts1, T_0to1, K0, K1)
            correct = epi_errs < 5e-4
            num_correct = np.sum(correct)
            precision = np.mean(correct) if len(correct) > 0 else 0
            matching_score = num_correct / len(kpts0) if len(kpts0) > 0 else 0

            thresh = 1.  # In pixels relative to resized image size.
            ret = estimate_pose(mkpts0, mkpts1, K0, K1, thresh)
            if ret is None:
                err_t, err_R = np.inf, np.inf
            else:
                R, t, inliers = ret
                err_t, err_R = compute_pose_error(T_0to1, R, t)

            # Write the evaluation results to disk.
            out_eval = {'error_t': err_t,
                        'error_R': err_R,
                        'precision': precision,
                        'matching_score': matching_score,
                        'num_correct': num_correct,
                        'epipolar_errors': epi_errs}
            np.savez(str(eval_path), **out_eval)
            timer.update('eval')

        if do_viz:
            # Visualize the matches.
            color = cm.jet(mconf)
            text = [
                'SuperGlue',
                'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
                'Matches: {}'.format(len(mkpts0)),
            ]
            if rot0 != 0 or rot1 != 0:
                text.append('Rotation: {}:{}'.format(rot0, rot1))

            # Display extra parameter info.
            k_thresh = matching.superpoint.config['keypoint_threshold']
            m_thresh = matching.superglue.config['match_threshold']
            small_text = [
                'Keypoint Threshold: {:.4f}'.format(k_thresh),
                'Match Threshold: {:.2f}'.format(m_thresh),
                'Image Pair: {}:{}'.format(stem0, stem1),
            ]

            make_matching_plot(
                image0, image1, kpts0, kpts1, mkpts0, mkpts1, color,
                text, viz_path, None,
                None, None, 'Matches', small_text)

            timer.update('viz_match')

        if do_viz_eval:
            # Visualize the evaluation results for the image pair.
            color = np.clip((epi_errs - 0) / (1e-3 - 0), 0, 1)
            color = error_colormap(1 - color)
            deg, delta = ' deg', 'Delta '
            if not None:
                deg, delta = '°', '$\\Delta$'
            e_t = 'FAIL' if np.isinf(err_t) else '{:.1f}{}'.format(err_t, deg)
            e_R = 'FAIL' if np.isinf(err_R) else '{:.1f}{}'.format(err_R, deg)
            text = [
                'SuperGlue',
                '{}R: {}'.format(delta, e_R), '{}t: {}'.format(delta, e_t),
                'inliers: {}/{}'.format(num_correct, (matches > -1).sum()),
            ]
            if rot0 != 0 or rot1 != 0:
                text.append('Rotation: {}:{}'.format(rot0, rot1))

            # Display extra parameter info (only works with --fast_viz).
            k_thresh = matching.superpoint.config['keypoint_threshold']
            m_thresh = matching.superglue.config['match_threshold']
            small_text = [
                'Keypoint Threshold: {:.4f}'.format(k_thresh),
                'Match Threshold: {:.2f}'.format(m_thresh),
                'Image Pair: {}:{}'.format(stem0, stem1),
            ]

            make_matching_plot(
                image0, image1, kpts0, kpts1, mkpts0,
                mkpts1, color, text, viz_eval_path,
                None, None,
                None, 'Relative Pose', small_text)

            timer.update('viz_eval')

        timer.print('Finished pair {:5} of {:5}'.format(i, len(pairs)))

 

Running inference on device "cpu"
Loaded SuperPoint model
test1
test2
test3
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "test"
Will write matches to directory "out"
Will write visualization images to directory "out"
[Finished pair     0 of     1] load_image=0.107 matcher=4.442 viz_match=0.858 total=5.407 sec {0.2 FPS} 


In [1]:
#! /usr/bin/env python3
#
# %BANNER_BEGIN%
# ---------------------------------------------------------------------
# %COPYRIGHT_BEGIN%
#
#  Magic Leap, Inc. ("COMPANY") CONFIDENTIAL
#
#  Unpublished Copyright (c) 2020
#  Magic Leap, Inc., All Rights Reserved.
#
# NOTICE:  All information contained herein is, and remains the property
# of COMPANY. The intellectual and technical concepts contained herein
# are proprietary to COMPANY and may be covered by U.S. and Foreign
# Patents, patents in process, and are protected by trade secret or
# copyright law.  Dissemination of this information or reproduction of
# this material is strictly forbidden unless prior written permission is
# obtained from COMPANY.  Access to the source code contained herein is
# hereby forbidden to anyone except current COMPANY employees, managers
# or contractors who have executed Confidentiality and Non-disclosure
# agreements explicitly covering such access.
#
# The copyright notice above does not evidence any actual or intended
# publication or disclosure  of  this source code, which includes
# information that is confidential and/or proprietary, and is a trade
# secret, of  COMPANY.   ANY REPRODUCTION, MODIFICATION, DISTRIBUTION,
# PUBLIC  PERFORMANCE, OR PUBLIC DISPLAY OF OR THROUGH USE  OF THIS
# SOURCE CODE  WITHOUT THE EXPRESS WRITTEN CONSENT OF COMPANY IS
# STRICTLY PROHIBITED, AND IN VIOLATION OF APPLICABLE LAWS AND
# INTERNATIONAL TREATIES.  THE RECEIPT OR POSSESSION OF  THIS SOURCE
# CODE AND/OR RELATED INFORMATION DOES NOT CONVEY OR IMPLY ANY RIGHTS
# TO REPRODUCE, DISCLOSE OR DISTRIBUTE ITS CONTENTS, OR TO MANUFACTURE,
# USE, OR SELL ANYTHING THAT IT  MAY DESCRIBE, IN WHOLE OR IN PART.
#
# %COPYRIGHT_END%
# ----------------------------------------------------------------------
# %AUTHORS_BEGIN%
#
#  Originating Authors: Paul-Edouard Sarlin
#                       Daniel DeTone
#                       Tomasz Malisiewicz
#
# %AUTHORS_END%
# --------------------------------------------------------------------*/
# %BANNER_END%

from pathlib import Path
import argparse
import random
import numpy as np
import matplotlib.cm as cm
import torch


from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

torch.set_grad_enabled(False)

info_pairs = 'test.txt'
input_dir = 'test'

with open(info_pairs, 'r') as f:
    pairs = [l.split() for l in f.readlines()]

device = 'cuda' if torch.cuda.is_available() and not None else 'cpu'
print('Running inference on device \"{}\"'.format(device))

config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024
    },
    'superglue': {
        'weights': 'outdoor',
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
        'descriptor_dim': 4,
        'keypoint_encoder': [32, 64, 128, 256],
    }
}

matching = Matching(config).eval().to(device)

Running inference on device "cpu"
Loaded SuperPoint model
test1
test2
test3
Loaded SuperGlue model ("outdoor" weights)


In [3]:
import torch

torch.tensor([4057.8595,  221.4774, 2.2358, 30.6575, 4])

tensor([4.0579e+03, 2.2148e+02, 2.2358e+00, 3.0657e+01, 4.0000e+00])

In [ ]:
keypoints0 = torch.tensor([980.7389, 618.5122,], [954.4825, 84.2555],  [1054.4825, 834.2555])
descriptors0 = torch.tensor([41126.0000,117.7602,1.3902, 24.2219,0], [307.8595,21.4774,5.2358,20.6575,2], [37.8595,333.4774,52.2358,21.6575,3])
scores0 = torch.tensor([1,1,1])

keypoints1 = torch.tensor([591.1545, 190.1858], [54.4825, 184.2555],  [654.4825, 334.2555])
descriptors1 = torch.tensor([1311.0000,119.4390,1.6900,23.2598,0], [607.8595,21.4774,5.2358,520.6575,2], [237.8595,456.4774,52.2358,1221.6575,1])
scores1 = torch.tensor([1,1,1])

In [ ]:
keypoints0 = torch.tensor([[980.7389, 618.5122,], [954.4825, 84.2555],  [1054.4825, 834.2555]])
descriptors0 = torch.tensor([[4.112600e+04, 3.078595e+02, 3.785950e+01],[117.7602,  21.4774, 333.4774],[ 1.3902,  5.2358, 52.2358],[24.2219, 20.6575, 21.6575],[0., 2., 3.]])
scores0 = torch.tensor([1,1,1])

keypoints1 = torch.tensor([[591.1545, 190.1858], [54.4825, 184.2555],  [654.4825, 334.2555]])
descriptors1 = torch.tensor([[1311.    ,  607.8595,  237.8595],[119.439 ,  21.4774, 456.4774],[ 1.69  ,  5.2358, 52.2358],[  23.2598,  520.6575, 1221.6575],[0., 2., 1.]])
scores1 = torch.tensor([1,1,1])

test_data = {'keypoints0': [keypoints0], 'scores0':[scores0], 'descriptors0':[descriptors0],'keypoints1': [keypoints1], 'scores1':[scores1], 'descriptors1':[descriptors1]}

In [18]:
import numpy as np
x = np.array([[[41126.0000,117.7602,1.3902, 24.2219,0], [307.8595,21.4774,5.2358,20.6575,2], [37.8595,333.4774,52.2358,21.6575,3]]])
[[x[:,:,0], x[:,:,1],x[:,:,2],x[:,:,3], x[:,:,4]]]


[[array([[4.112600e+04, 3.078595e+02, 3.785950e+01]]),
  array([[117.7602,  21.4774, 333.4774]]),
  array([[ 1.3902,  5.2358, 52.2358]]),
  array([[24.2219, 20.6575, 21.6575]]),
  array([[0., 2., 3.]])]]

In [11]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(5, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax(dim=1)  # Specify dim for Softmax

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

# Your artificial input data
input_data = torch.tensor([[41126.0000, 117.7602, 1.3902, 24.2219, 0],
                          [307.8595, 21.4774, 5.2358, 20.6575, 2],
                          [37.8595, 333.4774, 52.2358, 21.6575, 3]])

# Perform a forward pass
output = tinymodel(input_data)

# Now, 'output' contains the model's predictions (logits) for the provided input data

# If you want to apply softmax to get probabilities, you can do it like this:
softmax_output = torch.nn.functional.softmax(output, dim=1)
# Save the model to a file
torch.save(tinymodel.state_dict(), 'tinymodel.pth')

In [1]:
#! /usr/bin/env python3
#
# %BANNER_BEGIN%
# ---------------------------------------------------------------------
# %COPYRIGHT_BEGIN%
#
#  Magic Leap, Inc. ("COMPANY") CONFIDENTIAL
#
#  Unpublished Copyright (c) 2020
#  Magic Leap, Inc., All Rights Reserved.
#
# NOTICE:  All information contained herein is, and remains the property
# of COMPANY. The intellectual and technical concepts contained herein
# are proprietary to COMPANY and may be covered by U.S. and Foreign
# Patents, patents in process, and are protected by trade secret or
# copyright law.  Dissemination of this information or reproduction of
# this material is strictly forbidden unless prior written permission is
# obtained from COMPANY.  Access to the source code contained herein is
# hereby forbidden to anyone except current COMPANY employees, managers
# or contractors who have executed Confidentiality and Non-disclosure
# agreements explicitly covering such access.
#
# The copyright notice above does not evidence any actual or intended
# publication or disclosure  of  this source code, which includes
# information that is confidential and/or proprietary, and is a trade
# secret, of  COMPANY.   ANY REPRODUCTION, MODIFICATION, DISTRIBUTION,
# PUBLIC  PERFORMANCE, OR PUBLIC DISPLAY OF OR THROUGH USE  OF THIS
# SOURCE CODE  WITHOUT THE EXPRESS WRITTEN CONSENT OF COMPANY IS
# STRICTLY PROHIBITED, AND IN VIOLATION OF APPLICABLE LAWS AND
# INTERNATIONAL TREATIES.  THE RECEIPT OR POSSESSION OF  THIS SOURCE
# CODE AND/OR RELATED INFORMATION DOES NOT CONVEY OR IMPLY ANY RIGHTS
# TO REPRODUCE, DISCLOSE OR DISTRIBUTE ITS CONTENTS, OR TO MANUFACTURE,
# USE, OR SELL ANYTHING THAT IT  MAY DESCRIBE, IN WHOLE OR IN PART.
#
# %COPYRIGHT_END%
# ----------------------------------------------------------------------
# %AUTHORS_BEGIN%
#
#  Originating Authors: Paul-Edouard Sarlin
#                       Daniel DeTone
#                       Tomasz Malisiewicz
#
# %AUTHORS_END%
# --------------------------------------------------------------------*/
# %BANNER_END%

from pathlib import Path
import argparse
import random
import numpy as np
import matplotlib.cm as cm
import torch


from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

torch.set_grad_enabled(False)
from models import superglue_eval as superglue
#info_pairs = '/Users/damiangorcak/Desktop/dp/dp-satelites/test_graph_matching/SuperGluePretrainedNetwork-master/test.txt'
#input_dir = '/Users/damiangorcak/Desktop/dp/dp-satelites/test_graph_matching/SuperGluePretrainedNetwork-master/test'

#with open(info_pairs, 'r') as f:
#    pairs = [l.split() for l in f.readlines()]

device = 'cuda' if torch.cuda.is_available() and not None else 'cpu'
print('Running inference on device \"{}\"'.format(device))

config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024
    },
    'superglue': {
        'weights': 'my',
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
        'descriptor_dim': 5,
        'keypoint_encoder': [1, 2, 4, 5],
    }
}

#matching = Matching(config).eval().to(device)


timer = AverageTimer(newline=True)


keypoints0 = torch.tensor([[980.7389, 618.5122,], [954.4825, 84.2555],  [1054.4825, 834.2555]])
descriptors0 = torch.tensor([[4.112600e+04, 3.078595e+02, 3.785950e+01],[117.7602,  21.4774, 333.4774],[ 1.3902,  5.2358, 52.2358],[24.2219, 20.6575, 21.6575],[0., 2., 3.]])
scores0 = torch.tensor([1,1,1])

keypoints1 = torch.tensor([[591.1545, 190.1858], [54.4825, 184.2555],  [654.4825, 334.2555]])
descriptors1 = torch.tensor([[1311.    ,  607.8595,  237.8595],[119.439 ,  21.4774, 456.4774],[ 1.69  ,  5.2358, 52.2358],[  23.2598,  520.6575, 1221.6575],[0., 2., 1.]])
scores1 = torch.tensor([1,1,1])

pairs = {'keypoints0': [keypoints0], 'scores0':[scores0], 'descriptors0':[descriptors0],'keypoints1': [keypoints1], 'scores1':[scores1], 'descriptors1':[descriptors1]}



superglue = superglue.SuperGlue(config.get('superglue', {}))
#pred = matching({'image0': inp0, 'image1': inp1})
#pred = {k: v[0].cpu().numpy() for k, v in pred.items()}


pred  = superglue(pairs)
#### try this to change with our features
#kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
#matches, conf = pred['matches0'], pred['matching_scores0']


ModuleNotFoundError: No module named 'numpy'

In [7]:
import numpy as np
import torch
from models.utils import (AverageTimer)
import pickle
import numpy as np
from models import superglue_eval as superglue

torch.set_grad_enabled(False)



device = 'cuda' if torch.cuda.is_available() and not None else 'cpu'
print('Running inference on device \"{}\"'.format(device))

config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': 1024
    }
}

def normalize_array(arr):
    """
    Normalize an array to have values between 0 and 1.
    
    Args:
    arr (numpy.ndarray): Input array.

    Returns:
    numpy.ndarray: Normalized array.
    """
    min_val = np.min(arr)
    max_val = np.max(arr)
    normalized_arr = (arr - min_val) / (max_val - min_val)
    return normalized_arr

#matching = Matching(config).eval().to(device)


train_data = []

sc = [1 for i in range(0,10)]
with open('../data/data_1.pkl', 'rb') as fp:
        data = pickle.load(fp)

tmp = np.array([[x,x] for x in range(0, 15)])
all_matches = np.concatenate([
tmp,
np.zeros((15,1), dtype=np.int64),  # This is effectively adding nothing
np.zeros((15,1), dtype=np.int64)   # This is effectively adding nothing
], axis=1)


for i in range(0,len(data)):
    data[i]['keypoints0'] = [tuple(row) for row  in np.array(data[i]['keypoints0'])]
    data[i]['keypoints1'] = [tuple(row) for row in  np.array(data[i]['keypoints1'])]

    descs0 = np.array([normalize_array(x) for x in np.array(data[i]['descriptors0'][:-1])])
    data[i]['descriptors0'] = descs0.T.reshape(15,1500,5)
    descs1 = np.array([normalize_array(x) for x in np.array(data[i]['descriptors1'][:-1])])
    data[i]['descriptors1'] = descs1.T.reshape(15,1500,5)

    for j in range(0,1500):
        kp0 = []
        kp1 = []
        desc0 = []
        desc1 = []

        for k in range(0,15):

            kp0.append(data[i]['keypoints0'][k][j])
            kp1.append(data[i]['keypoints1'][k][j])

            desc0.append(data[i]['descriptors0'][k][j])
            desc1.append(data[i]['descriptors1'][k][j])
        

        kp0 =  [v for i,v in enumerate(kp0) if i not in frozenset((0,12,13,14,1))] 

        kp1 =  [v for i,v in enumerate(kp1) if i not in frozenset((0,12,13,14,1))] 

        desc0 =  [v for i,v in enumerate(desc0) if i not in frozenset((0,12,13,14,1))] 

        desc1 =  [v for i,v in enumerate(desc1) if i not in frozenset((0,12,13,14,1))] 


        

        train_data.append({'keypoints0': kp0, 'keypoints1': kp1, 'descriptors0': desc0,'descriptors1': desc1, 'scores0':sc, 'scores1': sc})

timer = AverageTimer(newline=True)

pairs = train_data[9946]

# Store data (serialize)
with open('test_data.pkl', 'wb') as handle:
    pickle.dump(pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)




Running inference on device "cpu"
